In [18]:
CODE_DIR = '/home/shreyas/Documents/git/wheelai/'
traindata_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtaV/sample_train/'
validdata_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtaV/valid/'
results_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtaV/results/'

In [40]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model,  Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.applications import VGG16
from keras.layers import Lambda, Cropping2D, Activation, ELU

%matplotlib inline



### Finetune and Train

In [20]:
batch_size = 64
epochs = 3

In [21]:
def get_batches(path, class_mode='categorical', gen=image.ImageDataGenerator(), \
                shuffle=True, target_size=(160, 320), batch_size=1):
    return gen.flow_from_directory(path, class_mode=class_mode, batch_size=batch_size, \
                                   target_size=target_size, shuffle=shuffle)

def get_steps(batches, batch_size):
    steps = int(batches.samples/batch_size)
    return (steps if batches.samples%batch_size==0 else (steps+1))


In [22]:
train_b = get_batches(traindata_path, batch_size=batch_size)
valid_b = get_batches(validdata_path, batch_size=batch_size)

Found 12954 images belonging to 3 classes.
Found 3375 images belonging to 3 classes.


In [23]:
train_steps = get_steps(train_b, batch_size)
valid_steps = get_steps(valid_b, batch_size)
num_class = train_b.num_class

In [24]:
train_labels = train_b.classes
valid_labels = valid_b.classes

In [25]:
y_train = keras.utils.to_categorical(train_labels)
y_valid = keras.utils.to_categorical(valid_labels)

In [26]:
print(train_labels.shape, valid_labels.shape, y_train.shape, y_valid.shape)

(12954,) (3375,) (12954, 3) (3375, 3)


In [27]:
trn_b = get_batches(traindata_path, class_mode=None, shuffle=False, batch_size=batch_size)
val_b = get_batches(validdata_path, class_mode=None, shuffle=False, batch_size=batch_size)

Found 12954 images belonging to 3 classes.
Found 3375 images belonging to 3 classes.


In [28]:
gen_augdata = image.ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, 
                                      shear_range=0.2, zoom_range=0.2)
augtrain_b = get_batches(traindata_path, gen=gen_augdata, class_mode='categorical',\
                         batch_size=batch_size)

Found 12954 images belonging to 3 classes.


In [29]:
#from keras.applications.resnet50 import ResNet50
#from keras.preprocessing import image
#from keras.applications.resnet50 import preprocess_input, decode_predictions

#def resnet50():
#    model = ResNet50(include_top = False, weights='imagenet', input_shape=(200, 400, 3))
    

In [49]:
filter_size = 3
pool_size = (2,2)


def get_model():
    model = Sequential()
    model.add(Cropping2D(cropping=((70,0), (0,0)), input_shape=(160, 320, 3)))
    model.add(Lambda(lambda x: x/255.-0.5))
    model.add(Conv2D(3,1,1,
                        border_mode='valid',
                        name='conv0', init='he_normal'))
    model.add(Conv2D(32,filter_size,filter_size,
                        border_mode='valid',
                        name='conv1', init='he_normal'))
    model.add(ELU())
    model.add(Conv2D(32,filter_size,filter_size,
                        border_mode='valid',
                        name='conv2', init='he_normal'))
    model.add(ELU())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.5))
    model.add(Conv2D(64,filter_size,filter_size,
                        border_mode='valid',
                        name='conv3', init='he_normal'))
    model.add(ELU())

    model.add(Conv2D(64,filter_size,filter_size,
                        border_mode='valid',
                        name='conv4', init='he_normal'))
    model.add(ELU())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.5))
    model.add(Conv2D(128,filter_size,filter_size,
                        border_mode='valid',
                        name='conv5', init='he_normal'))
    model.add(ELU())
    model.add(Conv2D(128,filter_size,filter_size,
                        border_mode='valid',
                        name='conv6', init='he_normal'))
    model.add(ELU())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(512,name='hidden1', init='he_normal'))
    model.add(ELU())
    model.add(Dropout(0.5))
    model.add(Dense(64,name='hidden2', init='he_normal'))
    model.add(ELU())
    model.add(Dropout(0.5))
    model.add(Dense(16,name='hidden3',init='he_normal'))
    model.add(ELU())
    model.add(Dropout(0.5))
    model.add(Dense(3, name='output', init='he_normal'))
    return model

In [50]:
model = get_model()
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='categorical_crossentropy')

/home/shreyas/anaconda3/envs/wheelai/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (1, 1), padding="valid", name="conv0", kernel_initializer="he_normal")`
/home/shreyas/anaconda3/envs/wheelai/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="valid", name="conv1", kernel_initializer="he_normal")`
/home/shreyas/anaconda3/envs/wheelai/lib/python3.5/site-packages/ipykernel/__main__.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="valid", name="conv2", kernel_initializer="he_normal")`
/home/shreyas/anaconda3/envs/wheelai/lib/python3.5/site-packages/ipykernel/__main__.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid", name="conv3", kernel_initializer="he_normal")`
/home/shreyas/anaconda3/envs/wheelai/lib/python3.5/site-packages/

In [51]:
filepath1 = results_path+'nvidia.h5'
checkpoint1 = ModelCheckpoint(filepath1, monitor='val_loss', verbose=1, save_best_only=False,\
                             save_weights_only=True, mode='min', period=1)
callbacks1=[checkpoint1]

In [52]:
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=5, callbacks=callbacks1,\
                        validation_data=valid_b, validation_steps=valid_steps)

Epoch 1/5
203/203 [==============================] - 499s - loss: 3.3706 - val_loss: 1.1652
Epoch 2/5
203/203 [==============================] - 167s - loss: 2.9268 - val_loss: 1.1566
Epoch 3/5
203/203 [==============================] - 168s - loss: 2.5422 - val_loss: 1.1358
Epoch 4/5
203/203 [==============================] - 167s - loss: 2.5560 - val_loss: 1.0897
Epoch 5/5
203/203 [==============================] - 168s - loss: 2.3597 - val_loss: 1.0907


In [53]:
filepath2 = results_path+'nvidia_aug.h5'
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1, save_best_only=False,\
                             save_weights_only=True, mode='min', period=1)
callbacks2=[checkpoint2]

In [54]:
model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=8, callbacks=callbacks2,\
                        validation_data=valid_b, validation_steps=valid_steps)

Epoch 1/8
203/203 [==============================] - 234s - loss: 2.2824 - val_loss: 1.0971
Epoch 2/8
203/203 [==============================] - 223s - loss: 2.2388 - val_loss: 1.1082
Epoch 3/8
203/203 [==============================] - 224s - loss: 2.2156 - val_loss: 1.1135
Epoch 4/8
203/203 [==============================] - 223s - loss: 2.0604 - val_loss: 1.1320
Epoch 5/8
203/203 [==============================] - 225s - loss: 1.9741 - val_loss: 1.1575
Epoch 6/8
203/203 [==============================] - 224s - loss: 1.9506 - val_loss: 1.1193
Epoch 7/8
203/203 [==============================] - 223s - loss: 1.9495 - val_loss: 1.1244
Epoch 8/8
203/203 [==============================] - 223s - loss: 1.9840 - val_loss: 1.1220


In [55]:
model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=3, callbacks=callbacks2,\
                        validation_data=valid_b, validation_steps=valid_steps)

Epoch 1/3
203/203 [==============================] - 171s - loss: 1.8784 - val_loss: 1.1216
Epoch 2/3
203/203 [==============================] - 167s - loss: 1.8446 - val_loss: 1.1324
Epoch 3/3
203/203 [==============================] - 165s - loss: 1.8274 - val_loss: 1.1256


In [ ]:
# serialize model to JSON
model_path = results_path+'nvidia.json'
model_path = results_path+'nvidia.json'
model_json = model.to_json()
with open(model_path, "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(filepath2)
print("Saved model to disk")